In [1]:
import random
random.seed(42)
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import os

sns.set()

df = pd.read_csv("registro01.csv",)
df.columns = ['x0','x1','x2','x3','x4','x5','x6',
              'x7','x8','x9', 'target']

#exibir valores ausentes ou null
df.isnull().sum().sort_values(ascending=False)[:10]
print("Número de linhas e colunas no conjunto de treinamento:", df.shape)
attributes = list(df.columns)
#removendo valores nulos
df.dropna()

#preencher os nulos
df.fillna(df.mean(0))

#remover duplicados
df.drop_duplicates()


Número de linhas e colunas no conjunto de treinamento: (1000, 11)


,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,target
0,-0.640731,0.677929,-0.285153,0.469127,1.629361,-0.407575,0.677929,0.781634,-0.132065,-0.125852,238.342700
1,-1.257229,0.199883,0.369755,0.905828,2.199284,0.407064,0.199883,-0.276165,0.136314,-0.835977,30.823609
2,-1.351330,-0.159821,-0.667650,-0.067187,1.309160,-0.702359,-0.159821,-0.133427,-0.683428,-1.016574,-135.707090
3,-2.432787,-0.996849,-0.704290,-0.750186,1.066865,-0.082292,-0.996849,-1.860141,-1.151661,-1.850721,-579.983211
4,-1.475213,-0.318454,-0.682221,-0.151922,1.148624,-0.583869,-0.318454,-0.621999,-0.619677,-1.134553,-250.729762
5,-2.036118,-1.078599,-0.499542,-0.172896,1.229001,-0.398228,-1.078599,-1.428676,-0.831903,-1.701761,-505.959443
6,-1.691544,-0.526730,-1.080969,-0.458262,0.553429,-1.145279,-0.526730,-0.684251,-1.480774,-0.981174,-331.728587
7,-0.441204,0.624858,0.518443,0.884419,2.270696,0.458040,0.624858,-0.164600,0.381948,-0.118522,171.255615
8,-2.648067,-1.897761,-1.109478,-1.327286,0.566497,-0.719584,-1.897761,-2.537705,-1.663454,-2.328546,-906.764602
9,-0.355480,0.576553,0.630524,1.034954,2.511433,0.900122,0.576553,-0.372276,0.410074,-0.076135,153.852732


In [2]:
reg = linear_model.LinearRegression()
reg.fit(df[['x0','x1','x2','x3','x4','x5',
              'x6','x7','x8','x9']], df['target'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [3]:
reg.coef_

array([ 31.3014815 ,  45.58207457,   2.43388278,  66.59024054,
         7.65963997,  38.6093196 ,  45.58207457, 161.07518436,
        16.8326167 ,  53.68336845])

In [4]:
reg.intercept_

29.269934525734897

In [5]:
#teste
reg.predict([[-0.640730831637,0.677929480838,-0.285153480842,
              0.469126570424,1.62936127945,-0.407574914906,
              0.677929480838,0.781634413421,-0.132065136609,
              -0.125852294603]])

array([215.22905653])